In [1]:
## Preamble: Package Loading
import numpy as np
import ipywidgets as ipw
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools as iter
import os
import datetime as dt
import json
import re
import kernel as kr
import regex
# Preamble working directory retreival

<h2> 1 Function Definitions </h2>

<h3> 1.1  Data Loading Function </h3>

In [22]:
def psc_load(input_filename):
    wkng_folder = os.getcwd()
    input_file_full = ''.join([wkng_folder,'/output/',input_filename])
    with open(input_file_full) as f_obj: 
        pscdata = json.load(f_obj)
    inpt_dic = pscdata[0]
    bt_rs = pscdata[1]
    bt_df = pd.DataFrame(bt_rs['beta_cntrd'] ).T
    bt_df.columns = bt_rs['beta_sum_row']
    bt_sm = pd.DataFrame(bt_rs['beta_sum_dat']).T
    bt_sm.columns = bt_rs['beta_sum_row']
    bt_sm.index = bt_rs['beta_sum_clmn']
    al_rs = pscdata[2]
    al_ldf = [pd.DataFrame(al_rs['alpha_cntrd'][j]).T  for j in range(len(al_rs['alpha_cntrd']))]
    #al_stupid = [ al_rs['alpha_sum_row'][:13] , al_rs['alpha_sum_row'][13] , al_rs['alpha_sum_row'][14]]
    al_sm = [pd.DataFrame(al_rs['alpha_sum_dat'][j]).T  for j in range(len(al_rs['alpha_sum_dat']))]
    for j in range(len(al_rs['alpha_sum_dat'])):
        #al_ldf[j].columns = al_stupid[j]
        al_ldf[j].columns = al_rs['alpha_sum_row'][j]
        #al_sm[j].columns = al_stupid[j] 
        al_sm[j].columns = al_rs['alpha_sum_row'][j]
        al_sm[j].index= al_rs['alpha_sum_clmn']

    out = [inpt_dic, bt_df, bt_sm, al_ldf, al_sm]
    return out

<h3> 1.2 Plotting Function </h3>

In [23]:
## Plotting Function
def coeffden(coeff,line_nms,x_lm,y_lm,c_h,w,n_eqn,s_eqn):
    nds = len(coeff)
    w = w-1
    s_eqn = s_eqn-1
    if n_eqn > 1:
        coeff = [coeff[i][s_eqn] for i in range(nds)]
    ncfs = coeff[0].shape[0]
    plt.close('all')
    a = [ np.sort(coeff[i].iloc[:,w].values,axis = 0) for i in range(nds) ] 
    h = [ c_h*ncfs**(-1/5)*np.std(a[i]) for i in range(nds)]
    aden = [ kr.mvden(a[i],a[i],h[i],9).reshape(ncfs,1) for i in range(nds) ]
    f,ax = plt.subplots()
    f.set_figheight(7)
    f.set_figwidth(15)
    ax.set_xlim((x_lm[0],x_lm[1]))
    ax.set_ylim((0,y_lm))
    for i in range(nds):
        ax.plot(a[i],aden[i])
    ax.legend(line_nms)
    ax.grid(which = 'both')
    ax.set_title(''.join(['Distribution of Estimated ',coeff[0].columns[w]]))
    plt.show()

<h3> 1.3 Interactive Plot and Tables Display Function </h3>

In [24]:
def tbl_dsp(tables,n_eqn,s_eqn,line_nms):
    s_eqn = s_eqn-1
    if n_eqn > 1:
        for j in range(len(tables)):
            display(tables[j][s_eqn])
            display(''.join(['Case ', str(j+1),':', line_nms[j]]))
    elif n_eqn == 1:
        for j in range(len(tables)):
            display(tables[j])
            display(''.join(['Case ', str(j+1),':', line_nms[j]]))

def cfs_dsp(coeff,tables,n_eqn,y_lm):
    
    box_hlayout = ipw.Layout(display='flex',flex_flow='row',align_items='stretch'
                             ,width='95%')
    
    if n_eqn == 1:
        coeff_sel = ipw.IntSlider( min = 1 , max = coeff[0].shape[1], value = 1, step = 1
                             , description = 'Coefficient:'
                              ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})
    elif n_eqn > 1: 
        coeff_sel = ipw.IntSlider( min = 1 , max = coeff[0][0].shape[1], value = 1, step = 1
                              , description = 'Coefficient:'
                              ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})

    eqn_sel = ipw.IntSlider( min = 1 , max = n_eqn, value = 1, step = 1, description = 'Equation:'
                              ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})

    ch_sel = ipw.FloatSlider(min = 0.1 , max = 3, value = 2.5, step =0.2, description = 'Bandwidth Constant'
                             ,width = 'auto',layout = box_hlayout, style = {'description_width': 'initial'})

    xlim_sel = ipw.FloatRangeSlider( value=[-0.4, 0.4], min=-0.5,max=0.5, step=0.05,description='x limits:'
                                    ,disabled=False ,continuous_update=False, orientation='horizontal'
                                    ,readout=True ,readout_format='.1f', width = 'auto'
                                    ,layout=box_hlayout, style = {'description_width': 'initial'}) 

    coeff_out =  ipw.interactive_output(coeffden ,{'coeff': ipw.fixed(coeff), 'line_nms': ipw.fixed(line_nms)
                                                 ,'x_lm': xlim_sel, 'y_lm': ipw.fixed(y_lm)
                                                 ,'c_h': ch_sel,'w': coeff_sel,'n_eqn': ipw.fixed(n_eqn)
                                                 ,'s_eqn': eqn_sel})  

    table_out = ipw.interactive_output(tbl_dsp,{'tables': ipw.fixed(tables),'n_eqn': ipw.fixed(n_eqn)
                                                ,'s_eqn': eqn_sel,'line_nms': ipw.fixed(line_nms)} )

    return ipw.VBox([table_out,coeff_out,coeff_sel,ch_sel,xlim_sel,eqn_sel])

<h3> 1.4 Input Dictionary Display Function </h3>

In [37]:
def indict_dsp(in_dcts):
    w_sel = ipw.IntSlider( min = 1 , max = len(in_dcts), value = 1, step = 1
                                 , description = 'Results Dataset:'
                                  ,width = 'auto', style = {'description_width': 'initial'})
    def in_dcts_dsp(in_dcts,w):
        display(in_dcts[w-1])

    dict_out = ipw.interactive_output(in_dcts_dsp,{'in_dcts': ipw.fixed(in_dcts),'w': w_sel })
    return ipw.VBox([w_sel,dict_out])

<h2> 2 Results </h2> 

In [38]:
inpt_filenames = ['pscout_6_12_1954.json' ,'pscout_6_12_1220.json' , 'pscout_6_12_1799.json']

res_out = [psc_load(inpt_filenames[i]) for i in range(len(inpt_filenames))]
in_dcts = [res_out[i][0] for i in range(len(inpt_filenames))]
dgp_filenames = [ in_dcts[i]['input_filename'] for i in range(len(inpt_filenames)) ]
bcoeff  = [res_out[i][1] for i in range(len(inpt_filenames))]
acoeff  = [res_out[i][3] for i in range(len(inpt_filenames))]
btables = [res_out[i][2] for i in range(len(inpt_filenames))]
atables = [res_out[i][4] for i in range(len(inpt_filenames))]

line_nms = ['n=30', 'n=50' ,'n=70']

In [39]:
indict_dsp(in_dcts)

In [26]:
display(cfs_dsp(bcoeff,btables,1,12))
display(cfs_dsp(acoeff,atables,2,5))